In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("Case Study - Deep Dive Analysis.xlsx")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280 entries, 0 to 1279
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Zone                   1280 non-null   object        
 1   Region                 1280 non-null   object        
 2   Manufacturer           1280 non-null   object        
 3   Brand                  1280 non-null   object        
 4   Subbrand               1280 non-null   object        
 5   Item                   1280 non-null   object        
 6   PackSize               1280 non-null   object        
 7   Packtype               1280 non-null   object        
 8   month                  1280 non-null   datetime64[ns]
 9   Value Offtake(000 Rs)  1280 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 100.1+ KB


In [4]:
df.head()

,Zone,Region,Manufacturer,Brand,Subbrand,Item,PackSize,Packtype,month,Value Offtake(000 Rs)
0,East,Rural,GLAXOSMITHKLINE,HORLICKS,HORLICKS LITE,HORLICKS LITE 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4883.050
1,East,Rural,GLAXOSMITHKLINE,HORLICKS,WOMENS HORLICKS,WOMENS HORLICKS 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4460.013
2,East,Rural,GLAXOSMITHKLINE,BOOST,BOOST SPORTS,BOOST SPORTS 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,3230.254
3,East,Rural,GLAXOSMITHKLINE,HORLICKS,HORLICKS HEALTH AND NUTRITION,HORLICKS HEALTH AND NUTRITION 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4548.961
4,East,Rural,GLAXOSMITHKLINE,BOOST,BOOST HEALTH AND ENERGY,BOOST HEALTH AND ENERGYÂ 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,2993.130


In [5]:
df.tail()

,Zone,Region,Manufacturer,Brand,Subbrand,Item,PackSize,Packtype,month,Value Offtake(000 Rs)
1275,West,Urban,NESTLE,MILO,MILO ACTIVE GO,MILO ACTIVE GO 400 GMS REFILL PACK,400 GMS,REFILL PACK,2019-05-01,954.106
1276,West,Urban,NESTLE,MILO,MILO ACTIVE GO,NESTLE_item_1,400 GMS,REFILL PACK,2019-05-01,361.682
1277,West,Urban,NESTLE,MILO,MILO ACTIVE GO,NESTLE_item_2,400 GMS,REFILL PACK,2019-05-01,352.389
1278,West,Urban,NESTLE,MILO,MILO ACTIVE GO,NESTLE_item_3,400 GMS,REFILL PACK,2019-05-01,320.917
1279,West,Urban,NESTLE,MILO,MILO ACTIVE GO,NESTLE_item_4,400 GMS,REFILL PACK,2019-05-01,245.041


##### Objective:
To create a generic Python function which can do a deep dive analysis and give out exact focus area which are behind the cause of drop of Value Sales for a particular Manufacturer

In [6]:
# helper function
def DataReModel(target_df, colname):
    target_subset = target_df.groupby(['Manufacturer', 'Zone','Region', 'Brand', 'Subbrand']).sum()
    target = target_subset.reset_index().melt(id_vars=['Manufacturer', 'Value Offtake(000 Rs)'], var_name='Level', value_name='Focus Area')
    target = target[['Manufacturer', 'Level', 'Focus Area', 'Value Offtake(000 Rs)']].groupby(['Manufacturer', 'Level', 'Focus Area']).mean().reset_index()
    target.columns = ['Manufacturer', 'Level', 'Focus Area', colname]
    return target

In [7]:
def Deep_Dive_Analysis(Manufacturer = "X", target_period = "May2019", reference_period = "Apr2019"):
    target_period = pd.Timestamp(target_period)
    reference_period = pd.Timestamp(reference_period)
    target_df = df[(df['Manufacturer'] == Manufacturer) & (df['month'] == target_period)]
    reference_df = df[(df['Manufacturer'] == Manufacturer) & (df['month'] == reference_period)]
    target_sales = target_df['Value Offtake(000 Rs)'].sum()
    reference_sales = reference_df['Value Offtake(000 Rs)'].sum()
    time_period = (target_period - reference_period)
    if target_sales > reference_sales:
        print("There is no drop in the sales for Manufacturer - {}  in the {} period".format(Manufacturer, time_period ))
        return
    else:
        target = DataReModel(target_df, 'Target Sales')
        reference = DataReModel(reference_df, 'Reference Sales')
        df_merge = pd.merge(target, reference, how='inner', on=['Manufacturer', 'Level', 'Focus Area'])
        df_merge["Growth_Rate"] = ((df_merge['Target Sales'] - df_merge['Reference Sales'])*100)/df_merge['Reference Sales']
        df_merge['Contribution'] = df_merge['Target Sales']* 100 / df_merge['Target Sales'].sum()
        df_merge['Product'] = df_merge["Growth_Rate"] * df_merge['Contribution']
        df_sorted = df_merge.sort_values('Contribution', ascending=False)
        final_df = df_sorted[['Manufacturer', 'Level', 'Focus Area', 'Growth_Rate', 'Contribution', 'Product']]
        return final_df

In [8]:
Deep_Dive_Analysis('GLAXOSMITHKLINE', 'Mar2019', 'Feb2019')

,Manufacturer,Level,Focus Area,Growth_Rate,Contribution,Product
6,GLAXOSMITHKLINE,Subbrand,HORLICKS HEALTH AND NUTRITION,-10.173930,16.390811,-166.758967
8,GLAXOSMITHKLINE,Subbrand,WOMENS HORLICKS,-7.806306,12.914953,-100.818080
1,GLAXOSMITHKLINE,Brand,HORLICKS,-10.400015,11.281056,-117.323153
3,GLAXOSMITHKLINE,Region,Urban,-9.337589,11.229779,-104.859061
10,GLAXOSMITHKLINE,Zone,North,-13.481339,10.749754,-144.921087
9,GLAXOSMITHKLINE,Zone,East,-13.274534,10.442482,-138.619084
12,GLAXOSMITHKLINE,Zone,West,-0.968220,8.933359,-8.649456
7,GLAXOSMITHKLINE,Subbrand,HORLICKS LITE,-17.735432,4.537404,-80.472824
2,GLAXOSMITHKLINE,Region,Rural,-13.390231,4.420670,-59.193793
5,GLAXOSMITHKLINE,Subbrand,BOOST SPORTS,-7.920156,3.052529,-24.176506
